# SQL 

# Оглавление <a name="шаг-0"></a>
   [Введение](#0)

   
1. [Загрузка данных](#1)
        
2. [Формирование запросов](#2)

    2.1. [Подсчёт количества  книг, которое  вышло после 1 января 2000 года](#2-1)
    
    2.2. [Для каждой книги подсчёт количества обзоров и средней оценки](#2-2)
    
    2.3. [Опредение издательства, которое выпустило наибольшее число книг толще 50 страниц — так будут исключены из анализа брошюры](#2-3)
    
    2.4. [Определение автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками](#2-4)
    
    2.5. [Подсчёт среднего количества обзоров от пользователей, которые поставили больше 50 оценок](#2-5)

    
3.  [Результаты](#3)

# Введение

**Целью**  исследования является подготовка ценностного предложения для нового продукта. В качестве нового продукта выступает крупный сервис для чтения книг по подписке

**Задачи исследования**
- Подсчёт количества  книг, которое  вышло после 1 января 2000 года;
- Для каждой книги подсчёт количества обзоров и средней оценки;
- Опредение издательства, которое выпустило наибольшее число книг толще 50 страниц — так будут исключены из анализа брошюры;
- Определение автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;
- Подсчёт среднего количества обзоров от пользователей, которые поставили больше 50 оценок.
 


Для анализа предоставлены таблицы:

**Таблица books**
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors**
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers**
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings**
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.
**Таблица reviews**
Содержит данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.


# 1. Загрузка данных<a name="1"></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', #пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:
query = '''SELECT *
           FROM books'''
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [4]:
query = '''SELECT *
           FROM authors'''
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [6]:
query = '''SELECT *
           FROM publishers'''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [8]:
query = '''SELECT *
           FROM ratings'''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [10]:
query = '''SELECT *
           FROM reviews'''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [11]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


# 2. Формирование запросов <a name="2"></a>

## 2.1. Подсчёт количества  книг, которое  вышло после 1 января 2000 года <a name="2-1"></a>

In [12]:
number_books = ''' SELECT COUNT (book_id) 
                   FROM books
                   WHERE publication_date > '2000-01-01'
                   '''   
number_books = pd.io.sql.read_sql(number_books, con = engine)
number_books

,count
0,819


## 2.2. Для каждой книги подсчёт количества обзоров и средней оценки<a name="2-2"></a>

In [13]:
rating_review = ''' SELECT title, AVG(rating) as average_rating, COUNT(DISTINCT review_id) AS count_review
                    FROM books
                    JOIN ratings on ratings.book_id = books.book_id
                    JOIN reviews on reviews.book_id = books.book_id
                    GROUP BY books.book_id
                    ORDER BY average_rating DESC, count_review DESC
                    --LIMIT 10'''
rating_review = pd.io.sql.read_sql(rating_review, con = engine)
rating_review.head()

,title,average_rating,count_review
0,A Dirty Job (Grim Reaper #1),5.0,4
1,School's Out—Forever (Maximum Ride #2),5.0,3
2,Moneyball: The Art of Winning an Unfair Game,5.0,3
3,Angels Fall,5.0,2
4,Women,5.0,2


In [14]:
rating_review['count_review'].sum()

2793

## 2.3. Опредение издательства, которое выпустило наибольшее число книг толще 50 страниц — так будут исключены из анализа брошюры<a name="2-3"></a>

In [15]:
large_editions  = '''SELECT  publisher, COUNT (book_id) AS books_count
                    FROM books  JOIN publishers ON books.publisher_id = publishers.publisher_id

                    WHERE num_pages >50
                    GROUP BY publisher   
                    ORDER BY books_count DESC
                    LIMIT 10'''
large_editions = pd.io.sql.read_sql(large_editions, con = engine)
large_editions

,publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


##  2.4. Определение автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками <a name="2-4"></a>

In [16]:
top_authors = ''' SELECT author, books.author_id, AVG(a.book_avg) AS rating_avg
        FROM books
        JOIN authors ON   books.author_id= authors.author_id
        JOIN
            (SELECT book_id, AVG(rating) AS book_avg
            FROM ratings
            GROUP BY book_id 
            HAVING COUNT(rating_id) >= 50
            ) AS a  ON  a.book_id = books.book_id
        
        GROUP BY author, books.author_id  
        ORDER BY rating_avg DESC
                '''
top_authors = pd.io.sql.read_sql(top_authors, con = engine)
top_authors

,author,author_id,rating_avg
0,J.K. Rowling/Mary GrandPré,236,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,402,4.264151
2,J.R.R. Tolkien,240,4.258446
3,Louisa May Alcott,376,4.192308
4,Rick Riordan,498,4.080645
5,William Golding,621,3.901408
6,J.D. Salinger,235,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,469,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,630,3.787879
9,Dan Brown,106,3.754540


##  2.5. Подсчёт среднего количества обзоров от пользователей, которые поставили больше 50 оценок<a name="2-5"></a>

In [17]:
average_text_rating = '''SELECT ratings.username, COUNT(ratings.rating_numb) as avg_num_rew
                         FROM
                            (SELECT username, COUNT (rating_id) AS rating_numb
                            FROM  ratings
                            GROUP BY username) AS ratings
                         JOIN reviews ON ratings.username = reviews.username
                         WHERE  ratings.rating_numb > 50
                         GROUP BY ratings.username
                         ORDER BY avg_num_rew DESC'''
average_text_rating = pd.io.sql.read_sql(average_text_rating, con = engine)
average_text_rating

,username,avg_num_rew
0,sfitzgerald,28
1,martinadam,27
2,richard89,26
3,jennifermiller,25
4,paul88,22
5,xdavis,18


In [18]:
average_text_rating['avg_num_rew'].mean()

24.333333333333332

# 3. Результаты

- Количество книг, вышедшее после 01.01.2020 составило 819.

- Наибольшее количество обзоров на книги с самым высоким средним рейтингом вышло на книги A Dirty Job (Grim Reaper #1) (рейтинг - 5, отзывов 4), School's Out—Forever (Maximum Ride #2)	(рейтинг - 5, отзывов 3) и Moneyball: The Art of Winning an Unfair Game	(рейтинг - 5, отзывов 3).

- Наибольшее число книг толще 50 страниц выпускают издательства Penguin Books- 42 книги, Vintage - 31 и  Grand Central Publishing - 25 книг. Издательство Penguin Classics стоит на 4м месте с 24 книгами.

- Тройку авторов с самой высокой средней оценкой книг, с учетом только книг с 50 и более оценками, составляют Джоан Роулинг, Маркус Зусак и  Джон Рональд Руэл Толкин.

- Анализ выявил всего 5 пользователей, которые поставили более 50 оценок: эти пользователи опубликовали от 18 до 28 обзоров. Таким образом, среднее количество обзоров от пользователей, которые поставили больше 50 оценок составляет  24,3 обзора.

- Как показывает анализ в новое книжное приложение должны быть добавлены книги в жанре фэнтези и детективном жанре. Наиболее популярны классические детективные  произведения. Основываясь на количестве отзывов, можно сказать, что чтение книг не выходит из моды. А предоставление авторам отзывов бонусов и приятных подарков позволит не только  стимулировать активность в приложении  и увеличивать лояльность пользователей, но и увеличить продажи. 